<figure>
<img src="latex/figs/Fig-Mod-Rankine-ENGRI-1120-F22.pdf" style="width:80%">
</figure>

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/ENGRI-1120-Prelim-1-F22`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-Prelim-1-F22/Project.toml`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-Prelim-1-F22/Manifest.toml`


In [2]:
using PrettyTables

### a) Compute the missing values in the table

In [3]:
# O₁ -
# From PH-diagram (or superheated table) -
H₁ = 446.1 # units: kJ/kg
S₁ = 1.75 # units: kJ/kg-K
T₁ = 80.0 # units C (given)
P₁ = 2.0; # units MPa (given)

In [4]:
# compute θ₂ -
# @ O₂ T = -50C P = 29.41 kPa
S₂ = 1.75 # units: kJ/kg-K
Sₗ = 0.7428
Sᵥ = 1.7819
θ₂ = (S₂ - Sₗ)/(Sᵥ - Sₗ)

0.9693003560773749

In [5]:
# O₂ -
T₂ = -50.0 # units C (from table or PH diagram)
P₂ = 29.41*(1/1000) # units MPa (given)

# compute H₂ -
# @ O₂ T = -50C P = 29.41 kPa
Hₗ = 136.1  # units: kJ/kg
Hᵥ = 368.0 # units: kJ/kg
H₂ = θ₂*Hᵥ+(1-θ₂)*Hₗ

360.8807525743432

In [6]:
# O₃ -
T₃ = -50.0 # units C (phase change, constant T)
P₃ = 29.41*(1/1000) # units MPa
θ₃ = 0.0 # all liquid after Condenser -

# Compute H₃ -
# H₃ is a satuared liquid with the S₃ = S₄ = Sₗ
H₃ = Hₗ
S₃ = 0.7428 # same at O₄ pump is isentropic units: kJ/kg-K

0.7428

In [7]:
# O₄ -
T₄ = T₃ # by assumption iii
P₄ = P₁ # constant P in boiler

# Compute H₄ -
# Problem assumption iii -
H₄ = H₃
S₄ = 0.7428 # Given in the problem units: kJ/kg-K

0.7428

In [8]:
# initialize -
number_of_op_points = 4
number_of_states = 6
state_table_data = Array{Any,2}(undef, number_of_op_points, number_of_states);

# setup some arrays -
H_array = [H₁, H₂, H₃, H₄];
S_array = [S₁, S₂, S₃, S₄];
T_array = [T₁, T₂, T₃, T₄];
P_array = [P₁, P₂, P₃, P₄];
θ_array = [1.0, θ₂, θ₃, 0.0];

# main loop -
for i ∈ 1:number_of_op_points
    
    for s ∈ 1:number_of_states
        
        if s == 1
            state_table_data[i,1] = i
        elseif s == 2
            state_table_data[i,2] = T_array[i]
        elseif s == 3
            state_table_data[i,3] = P_array[i]
        elseif s == 4
            state_table_data[i,4] = H_array[i]
        elseif s == 5
            state_table_data[i,5] = S_array[i]
        elseif s == 6
            state_table_data[i,6] = θ_array[i]
        end
    end
end

# setup header -
state_table_header = (["Oᵢ","T (C)","P (MPa)","H (kJ/kg)", "S (kJ/kg-K)", "θᵢ"]);

# make the table -
pretty_table(state_table_data; header=state_table_header)

┌────┬───────┬─────────┬───────────┬─────────────┬────────┐
│ Oᵢ │ T (C) │ P (MPa) │ H (kJ/kg) │ S (kJ/kg-K) │     θᵢ │
├────┼───────┼─────────┼───────────┼─────────────┼────────┤
│  1 │  80.0 │     2.0 │     446.1 │        1.75 │    1.0 │
│  2 │ -50.0 │ 0.02941 │   360.881 │        1.75 │ 0.9693 │
│  3 │ -50.0 │ 0.02941 │     136.1 │      0.7428 │    0.0 │
│  4 │ -50.0 │     2.0 │     136.1 │      0.7428 │    0.0 │
└────┴───────┴─────────┴───────────┴─────────────┴────────┘


### b) Compute heat and work values

In [18]:
# set stuff given in problem
ṁ = 2.25  # units kg/s
ηₜ = 0.85  # turbine efficiency 

0.85

In [11]:
# build connectivity array -
C = [
    1 -1 0 0 ; # 1 turbine
    0 1 -1 0 ; # 2 condenser
    0 0 1 -1 ; # 3 pump
    -1 0 0 1 ; # 4 boiler
];

# compute the right-hand side of the energy balance -
b = -ṁ*C*H_array

4-element Vector{Float64}:
 -191.7433067077278
 -505.75669329227225
    0.0
  697.5

##### check:

In [12]:
sum(b)

0.0

In [17]:
# setup Q̇_array -
# units: turbine (1), condenser (2), pump (3), boiler (4)
Q̇_array = [
    0.0  # 1 turbine
    b[2] # 2 condenser
    0.0  # 3 pump
    b[4] # 4 boiler
];

In [16]:
# setup Ẇ_array -
# units: turbine (1), condenser (2), pump (3), boiler (4)
Ẇ_array = [
    b[1]  # 1 turbine
    0.0   # 2 condenser
    b[3]  # 3 pump
    0.0   # 4 boiler
];

In [27]:
# build the heat and work table -
number_of_paths = 5 # includes the total row
number_of_cols = 4
heat_work_table_data = Array{Any,2}(undef, number_of_paths, number_of_cols)

# main loop -
for i ∈ 1:(number_of_paths - 1)
    
    for j ∈ 1:number_of_cols
        
        if j == 1
           heat_work_table_data[i,1] = i
        elseif j == 2
            heat_work_table_data[i,2] = Q̇_array[i]
        elseif j == 3
            heat_work_table_data[i,3] = Ẇ_array[i]
        elseif j == 4
            heat_work_table_data[i,4] = ηₜ*Ẇ_array[i]
        end
    end
end

# add total row -
heat_work_table_data[5,1] = "total"
heat_work_table_data[5,2] = sum(Q̇_array)
heat_work_table_data[5,3] = sum(Ẇ_array)
heat_work_table_data[5,4] = ηₜ*sum(Ẇ_array)

# header -
heat_work_table_header = (["Path i", "Q̇ᵢ (kJ/s)", "(ideal) Ẇᵢ (kJ/s)", "(actual) Ẇᵢ (kJ/s)"]);

# table -
pretty_table(heat_work_table_data; header=heat_work_table_header)

┌────────┬───────────┬───────────────────┬────────────────────┐
│ Path i │ Q̇ᵢ (kJ/s) │ (ideal) Ẇᵢ (kJ/s) │ (actual) Ẇᵢ (kJ/s) │
├────────┼───────────┼───────────────────┼────────────────────┤
│      1 │       0.0 │          -191.743 │           -162.982 │
│      2 │  -505.757 │               0.0 │                0.0 │
│      3 │       0.0 │               0.0 │                0.0 │
│      4 │     697.5 │               0.0 │                0.0 │
│  total │   191.743 │          -191.743 │           -162.982 │
└────────┴───────────┴───────────────────┴────────────────────┘


### c) Compute ideal Rankine cycle efficiency

In [29]:
η = -Ẇ_array[1]/Q̇_array[4]

0.27490079814728

### d) Compute ideal Rankine cycle efficiency with heat recycle

In [31]:
α = 0.33;
Q̇_R = abs(α*Q̇_array[2]) # rate of heat that we are going to recyle units: kJ/s

166.89970878644985

In [32]:
η_R = - Ẇ_array[1]/(Q̇_array[4] - Q̇_R)

0.3613705267088839

The efficiency goes up. We are getting the same amount of work out, but it costs less heat, hence an increase in the efficiency